In [49]:
from model import CNN_Encoder,RNN_Decoder
import tensorflow as tf
embedding_dim = 256 #임베딩 차원
units = 512 #노드 개수는 512개
vocab_size = 10000 + 1 #사전크기 50000개 

encoder = CNN_Encoder(embedding_dim)
decoder = RNN_Decoder(embedding_dim, units, vocab_size)

optimizer = tf.keras.optimizers.Adam()


#체크포인트 불러오기
checkpoint_path = "./checkpoints/final_model1"
ckpt = tf.train.Checkpoint(encoder=encoder,
                           decoder=decoder,
                           optimizer = optimizer)
ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

start_epoch = 0
if ckpt_manager.latest_checkpoint:
  start_epoch = int(ckpt_manager.latest_checkpoint.split('-')[-1])
  # restoring the latest checkpoint in checkpoint_path
  ckpt.restore(ckpt_manager.latest_checkpoint)



In [50]:
import tensorflow as tf
import pickle
image_model = tf.keras.applications.InceptionV3(include_top=False,weights='imagenet')
new_input = image_model.input
hidden_layer = image_model.layers[-1].output

image_features_extract_model = tf.keras.Model(new_input, hidden_layer)
    
with open('./tokenizer.pkl', 'rb') as f:
    tokenizer = pickle.load(f)
    
with open('./train_captions.pkl', 'rb') as f:
    train_captions = pickle.load(f)

In [51]:
def calc_max_length(tensor):
    return max(len(t) for t in tensor)

train_seqs = tokenizer.texts_to_sequences(train_captions)
cap_vector = tf.keras.preprocessing.sequence.pad_sequences(train_seqs, padding='post')
max_length = calc_max_length(train_seqs)

In [52]:
def load_image(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (299, 299))
    img = tf.keras.applications.inception_v3.preprocess_input(img)
    return img, image_path

def evaluate(image):
    hidden = decoder.reset_state(batch_size=1)

    temp_input = tf.expand_dims(load_image(image)[0], 0)
    img_tensor_val = image_features_extract_model(temp_input)
    img_tensor_val = tf.reshape(img_tensor_val, (img_tensor_val.shape[0], -1, img_tensor_val.shape[3]))

    features = encoder(img_tensor_val)

    dec_input = tf.expand_dims([tokenizer.word_index['<start>']], 0)
    result = []

    for i in range(max_length):
        predictions, hidden, attention_weights = decoder(dec_input, features, hidden)

        predicted_id = tf.random.categorical(predictions, 1)[0][0].numpy()
        result.append(tokenizer.index_word[predicted_id])

        if tokenizer.index_word[predicted_id] == '<end>':
            return result

        dec_input = tf.expand_dims([predicted_id], 0)

    return result

In [5]:
evaluate("/home/seeds/iconms12/tmp/tf_image_captioning/robber_img (5).jpg")

['a', 'child', 'is', 'sitting', 'in', 'front', 'of', 'a', 'suitcase', '<end>']

In [53]:
#번역
import requests
import re
import pandas as pd
import urllib.request
import urllib.parse
import ast

def trans(msg):
    client_id = "파파고API_Key"
    client_secret = "파파고secret"
    request_url = "https://openapi.naver.com/v1/papago/n2mt"
    headers = {"X-Naver-Client-Id": client_id , "X-Naver-Client-Secret": client_secret }
    response = requests.post(request_url, headers=headers, data={"source": "en", "target": "ja", "text": msg})
    response = requests.post(request_url, headers=headers, data={"source": "ja", "target": "ko", "text": response.json()['message']['result']['translatedText']})
    result = response.json()['message']['result']['translatedText']
    
    return result#(re.sub("[a-zA-Z]","",tmp).strip())

In [7]:
trans("apple")

'사과'

In [54]:
#전송
import sys
from sdk.api.message import Message
from sdk.exceptions import CoolsmsException
from datetime import datetime

##  @brief This sample code demonstrate how to send sms through CoolSMS Rest API PHP
def send_sms(msg):
    now= datetime.now()
    # set api key, api secret
    api_key = "Cool_sms_API_Key"
    api_secret = "Cool_sms_secret"

    ## 4 params(to, from, type, text) are mandatory. must be filled
    params = dict()
    params['LMS'] = 'sms' # Message type ( sms, lms, mms, ata )
    params['to'] = '송신자' # Recipients Number '01000000000,01000000001'
    params['from'] = '수신자' # Sender number
    dt_string = f"{now.year}년{now.month}월 {now.day}일 {now.hour:02}시 {now.minute:02}분 {now.second:02}초"
    result="현재위치: ㅁㅁㅁ 집의 CCTV \n 촬영시각: {} \n 상세내용: {}".format(dt_string,msg)
    params['text'] = result # Message

    cool = Message(api_key, api_secret)
    try:
        response = cool.send(params)
        print("Success Count : %s" % response['success_count'])
        print("Error Count : %s" % response['error_count'])
        print("Group ID : %s" % response['group_id'])

        if "error_list" in response:
            print("Error List : %s" % response['error_list'])

    except CoolsmsException as e:
        print("Error Code : %s" % e.code)
        print("Error Message : %s" % e.msg)

In [55]:
from socket import *
import struct
import shutil
import time
HOST = 'SERVER_IP'
PORT = 'SERVER_PORT'
ADDR = (HOST, PORT)
BUFF_SIZE = 1024
FILE_NAME = './webcam_transfer/clientFile.jpg'
src="/home/seeds/iconms12/tmp/tf_image_captioning/webcam_transfer/clientFile.jpg"
dst="/home/seeds/iconms12/tmp/tf_image_captioning/detect_annomaly/"
while True:
    try:
        serverSocket = socket(AF_INET, SOCK_STREAM)
        serverSocket.bind(ADDR)
        serverSocket.listen(5)

        print("접속대기중")
        clientSocket, addr = serverSocket.accept()
        print("접속완료")    
        count=1
        while True:
            FILE_SIZE = clientSocket.recv(8)
            print(FILE_SIZE)
            #ime.sleep(0.001)
            FILE_SIZE = FILE_SIZE.decode()
            FILE_LEN = 0
            f = open(FILE_NAME, 'wb')
            while True:
                client_file = clientSocket.recv(BUFF_SIZE)
                # print(client_file)

                if not client_file:
                    break

                f.write(client_file)
                FILE_LEN += len(client_file)

                if FILE_LEN == int(FILE_SIZE):
                    break
            f.close()
            print('client : ' + FILE_NAME + ' file transfer')
            image="/home/seeds/iconms12/tmp/tf_image_captioning/webcam_transfer/clientFile.jpg"
            server_msg = evaluate(image) #캡션 전송
            sms_msg=' '.join(server_msg[:-1])
            if 'knife' in server_msg or 'sword' in server_msg or 'scissors' in server_msg:
                shutil.copy2(src,dst+f"detect_knife_{count}.jpg")
                count+=1
                msg=trans(sms_msg)
                send_sms(msg)
            # server_msg = bytes(server_msg, encoding='utf-8')
            # clientSocket.send(server_msg)
            clientSocket.send(sms_msg.encode())
    except Exception as e:
        print(e)
        clientSocket.close()
        serverSocket.close()
        pass
        

접속대기중
접속완료
b''
client : ./webcam_transfer/clientFile.jpg file transfer
Expected image (JPEG, PNG, or GIF), got empty file [Op:DecodeJpeg]
[Errno 98] Address already in use
[Errno 98] Address already in use
[Errno 98] Address already in use
[Errno 98] Address already in use
[Errno 98] Address already in use
[Errno 98] Address already in use
[Errno 98] Address already in use
[Errno 98] Address already in use
[Errno 98] Address already in use
[Errno 98] Address already in use
접속대기중
접속완료
b'5279'
client : ./webcam_transfer/clientFile.jpg file transfer
b'13964'
client : ./webcam_transfer/clientFile.jpg file transfer
b'18818'
client : ./webcam_transfer/clientFile.jpg file transfer
b'31781'
client : ./webcam_transfer/clientFile.jpg file transfer
b'27884'
client : ./webcam_transfer/clientFile.jpg file transfer
b'30499'
client : ./webcam_transfer/clientFile.jpg file transfer
b'33816'
client : ./webcam_transfer/clientFile.jpg file transfer
b'31668'
client : ./webcam_transfer/clientFile.jpg file t

b'58297'
client : ./webcam_transfer/clientFile.jpg file transfer
b'46575'
client : ./webcam_transfer/clientFile.jpg file transfer
b'42052'
client : ./webcam_transfer/clientFile.jpg file transfer
b'43751'
client : ./webcam_transfer/clientFile.jpg file transfer
b'49286'
client : ./webcam_transfer/clientFile.jpg file transfer
b'68309'
client : ./webcam_transfer/clientFile.jpg file transfer
b'71960'
client : ./webcam_transfer/clientFile.jpg file transfer
b'73974'
client : ./webcam_transfer/clientFile.jpg file transfer
b'70829'
client : ./webcam_transfer/clientFile.jpg file transfer
b'71300'
client : ./webcam_transfer/clientFile.jpg file transfer
b'60037'
client : ./webcam_transfer/clientFile.jpg file transfer
b'50494'
client : ./webcam_transfer/clientFile.jpg file transfer
b'43978'
client : ./webcam_transfer/clientFile.jpg file transfer
b'27224'
client : ./webcam_transfer/clientFile.jpg file transfer
b'32843'
client : ./webcam_transfer/clientFile.jpg file transfer
b'35669'
client : ./webca

b'9170'
client : ./webcam_transfer/clientFile.jpg file transfer
b'8778'
client : ./webcam_transfer/clientFile.jpg file transfer
b'8915'
client : ./webcam_transfer/clientFile.jpg file transfer
b'8752'
client : ./webcam_transfer/clientFile.jpg file transfer
b'8924'
client : ./webcam_transfer/clientFile.jpg file transfer
b'8934'
client : ./webcam_transfer/clientFile.jpg file transfer
b'8565'
client : ./webcam_transfer/clientFile.jpg file transfer
b'8769'
client : ./webcam_transfer/clientFile.jpg file transfer
b'8899'
client : ./webcam_transfer/clientFile.jpg file transfer
b'8802'
client : ./webcam_transfer/clientFile.jpg file transfer
b'8912'
client : ./webcam_transfer/clientFile.jpg file transfer
b'9292'
client : ./webcam_transfer/clientFile.jpg file transfer
b'8884'
client : ./webcam_transfer/clientFile.jpg file transfer
b'9053'
client : ./webcam_transfer/clientFile.jpg file transfer
b'8738'
client : ./webcam_transfer/clientFile.jpg file transfer
b'8594'
client : ./webcam_transfer/clien

b'13865'
client : ./webcam_transfer/clientFile.jpg file transfer
b'13976'
client : ./webcam_transfer/clientFile.jpg file transfer
b'13958'
client : ./webcam_transfer/clientFile.jpg file transfer
b'14081'
client : ./webcam_transfer/clientFile.jpg file transfer
b'13726'
client : ./webcam_transfer/clientFile.jpg file transfer
b'13539'
client : ./webcam_transfer/clientFile.jpg file transfer
b'14516'
client : ./webcam_transfer/clientFile.jpg file transfer
b'14047'
client : ./webcam_transfer/clientFile.jpg file transfer
b'13871'
client : ./webcam_transfer/clientFile.jpg file transfer
b'13944'
client : ./webcam_transfer/clientFile.jpg file transfer
b'13711'
client : ./webcam_transfer/clientFile.jpg file transfer
b'13814'
client : ./webcam_transfer/clientFile.jpg file transfer
b'13426'
client : ./webcam_transfer/clientFile.jpg file transfer
b'13661'
client : ./webcam_transfer/clientFile.jpg file transfer
b'13165'
client : ./webcam_transfer/clientFile.jpg file transfer
b'13672'
client : ./webca

b'12459'
client : ./webcam_transfer/clientFile.jpg file transfer
b'12920'
client : ./webcam_transfer/clientFile.jpg file transfer
b'12329'
client : ./webcam_transfer/clientFile.jpg file transfer
b'11874'
client : ./webcam_transfer/clientFile.jpg file transfer
b'12051'
client : ./webcam_transfer/clientFile.jpg file transfer
b'12099'
client : ./webcam_transfer/clientFile.jpg file transfer
b'12681'
client : ./webcam_transfer/clientFile.jpg file transfer
b'13594'
client : ./webcam_transfer/clientFile.jpg file transfer
b'12343'
client : ./webcam_transfer/clientFile.jpg file transfer
b'13398'
client : ./webcam_transfer/clientFile.jpg file transfer
b'12511'
client : ./webcam_transfer/clientFile.jpg file transfer
b'12895'
client : ./webcam_transfer/clientFile.jpg file transfer
b'12413'
client : ./webcam_transfer/clientFile.jpg file transfer
b'13632'
client : ./webcam_transfer/clientFile.jpg file transfer
b'13488'
client : ./webcam_transfer/clientFile.jpg file transfer
b'13048'
client : ./webca

b'14179'
client : ./webcam_transfer/clientFile.jpg file transfer
b'14413'
client : ./webcam_transfer/clientFile.jpg file transfer
b'13814'
client : ./webcam_transfer/clientFile.jpg file transfer
b'14261'
client : ./webcam_transfer/clientFile.jpg file transfer
b'14053'
client : ./webcam_transfer/clientFile.jpg file transfer
b'13884'
client : ./webcam_transfer/clientFile.jpg file transfer
b'13802'
client : ./webcam_transfer/clientFile.jpg file transfer
b'8989'
client : ./webcam_transfer/clientFile.jpg file transfer
b'12913'
client : ./webcam_transfer/clientFile.jpg file transfer
b'9092'
client : ./webcam_transfer/clientFile.jpg file transfer
b'13463'
client : ./webcam_transfer/clientFile.jpg file transfer
b'14319'
client : ./webcam_transfer/clientFile.jpg file transfer
b'13831'
client : ./webcam_transfer/clientFile.jpg file transfer
b'13060'
client : ./webcam_transfer/clientFile.jpg file transfer
b'14118'
client : ./webcam_transfer/clientFile.jpg file transfer
b'13911'
client : ./webcam_

b'14358'
client : ./webcam_transfer/clientFile.jpg file transfer
b'14319'
client : ./webcam_transfer/clientFile.jpg file transfer
b'14290'
client : ./webcam_transfer/clientFile.jpg file transfer
b'14432'
client : ./webcam_transfer/clientFile.jpg file transfer
b'14193'
client : ./webcam_transfer/clientFile.jpg file transfer
b'14345'
client : ./webcam_transfer/clientFile.jpg file transfer
b'14375'
client : ./webcam_transfer/clientFile.jpg file transfer
b'13871'
client : ./webcam_transfer/clientFile.jpg file transfer
b'13301'
client : ./webcam_transfer/clientFile.jpg file transfer
b'13079'
client : ./webcam_transfer/clientFile.jpg file transfer
b'13328'
client : ./webcam_transfer/clientFile.jpg file transfer
b'13011'
client : ./webcam_transfer/clientFile.jpg file transfer
b'13898'
client : ./webcam_transfer/clientFile.jpg file transfer
b'13966'
client : ./webcam_transfer/clientFile.jpg file transfer
b'13193'
client : ./webcam_transfer/clientFile.jpg file transfer
b'13512'
client : ./webca

b'14388'
client : ./webcam_transfer/clientFile.jpg file transfer
b'14572'
client : ./webcam_transfer/clientFile.jpg file transfer
b'13845'
client : ./webcam_transfer/clientFile.jpg file transfer
b'14365'
client : ./webcam_transfer/clientFile.jpg file transfer
Success Count : 1
Error Count : 0
Group ID : R2GLrPEela54PVhQ
b'8634'
client : ./webcam_transfer/clientFile.jpg file transfer
b'7163'
client : ./webcam_transfer/clientFile.jpg file transfer
b'6921'
client : ./webcam_transfer/clientFile.jpg file transfer
b'7574'
client : ./webcam_transfer/clientFile.jpg file transfer
b'7788'
client : ./webcam_transfer/clientFile.jpg file transfer
b'7673'
client : ./webcam_transfer/clientFile.jpg file transfer
b'13185'
client : ./webcam_transfer/clientFile.jpg file transfer
b'13036'
client : ./webcam_transfer/clientFile.jpg file transfer
b'14116'
client : ./webcam_transfer/clientFile.jpg file transfer
b'13835'
client : ./webcam_transfer/clientFile.jpg file transfer
b'14190'
client : ./webcam_transfe

KeyboardInterrupt: 

In [56]:
clientSocket.close()
serverSocket.close()